<a href="https://colab.research.google.com/github/Fguarien/Analogique/blob/master/urllookup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Étape 1 : Importer les Bibliothèques Nécessaires

In [10]:
import pandas as pd
import requests
import json
import time
import logging
from itertools import islice


Étape 2 : Configurer les Logs

In [11]:
# Configurer les logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()


Étape 3 : Charger les URLs depuis un Fichier CSV et Remplacer les Wildcards

In [12]:
# Charger le fichier CSV
file_name = 'netskope_skopeit_alerts_08-26-2024.csv'  # Remplacez par le nom de votre fichier
logger.info("Chargement du fichier CSV.")
url_data = pd.read_csv(file_name)

# Suppression des doublons
url_data.drop_duplicates(subset='URL', inplace=True)
logger.info(f"Nombre d'URLs après suppression des doublons: {len(url_data)}")

# Remplacer les wildcards "*." par le domaine principal
def replace_wildcard(url):
    if url.startswith('*.'):
        return url.replace('*.', '')
    return url

url_data['URL'] = url_data['URL'].apply(replace_wildcard)
logger.info("Remplacement des wildcards par les domaines principaux terminé.")

# Extraire la liste des URLs
urls = url_data['URL'].tolist()


Étape 4 : Diviser les URLs en Lots de 100

In [13]:
def chunk_urls(url_list, chunk_size=100):
    """Diviser la liste des URLs en morceaux de taille chunk_size."""
    for i in range(0, len(url_list), chunk_size):
        yield url_list[i:i + chunk_size]

# Diviser les URLs en lots de 100
url_chunks = list(chunk_urls(urls, 100))
logger.info(f"Nombre de lots d'URLs à traiter: {len(url_chunks)}")


Étape 5 : Définir la Fonction de Requête API avec Gestion des Erreurs et Limite de Requêtes

In [31]:
# Adresse API de Netskope
api_url = "https://sitb-mivb.eu.goskope.com/api/v2/nsiq/urllookup"

# Remplacez par votre token d'autorisation
api_token = '1aebabf6eb47804ae59ddaaafc23d101'

def get_url_categories(urls):
    """Envoyer une requête à l'API Netskope pour obtenir les catégories des URLs."""
    headers = {
      'accept': 'application/json',
      'Content-Type': 'application/json',
      'Netskope-Api-Token': api_token  # Ajout du token d'autorisation
    }
    data = {
        "query": {
          "disable_dns_lookup": True,
          "urls": urls
  }
    }

    try:
        response = requests.post(api_url, headers=headers, json=data, timeout=10)

        if response.status_code == 200:
            logger.info(f"Requête réussie pour {len(urls)} URLs.")
            return response.json()  # Renvoie la réponse en JSON
        else:
            logger.error(f"Erreur lors de la requête : {response.status_code} - {response.text}")
            return None

    except requests.exceptions.RequestException as e:
        logger.error(f"Exception lors de la requête : {e}")
        return None


Étape 6 : Envoyer les Requêtes et Collecter les Résultats avec Limite de 4 Requêtes par Seconde

In [32]:
all_results = []

for i, chunk in enumerate(url_chunks):
    result = get_url_categories(chunk)
    if result:
        all_results.extend(result)  # Ajouter les résultats au tableau global

    logger.info(f"Traitement du lot {i+1}/{len(url_chunks)} terminé.")

    # Limiter à 4 requêtes par seconde
    if i % 4 == 3:  # Après chaque groupe de 4 requêtes
        time.sleep(1.5)  # Attendre 1.5 secondes avant de continuer
    else:
        time.sleep(0.25)  # Attendre 0.25 seconde pour les autres requêtes


ERROR:root:Exception lors de la requête : HTTPSConnectionPool(host='sitb-mivb.eu.goskope.com', port=443): Max retries exceeded with url: /api/v2/nsiq/urllookup (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x78efecc50220>: Failed to resolve 'sitb-mivb.eu.goskope.com' ([Errno -2] Name or service not known)"))
ERROR:root:Exception lors de la requête : HTTPSConnectionPool(host='sitb-mivb.eu.goskope.com', port=443): Max retries exceeded with url: /api/v2/nsiq/urllookup (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x78efecc53af0>: Failed to resolve 'sitb-mivb.eu.goskope.com' ([Errno -2] Name or service not known)"))
ERROR:root:Exception lors de la requête : HTTPSConnectionPool(host='sitb-mivb.eu.goskope.com', port=443): Max retries exceeded with url: /api/v2/nsiq/urllookup (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x78efea87c580>: Failed to resolve 'sitb-mivb.eu.goskope.com' ([Errno -2] Name

Étape 7 : Sauvegarder les Résultats

In [ ]:
# Convertir les résultats en DataFrame
results_df = pd.DataFrame(all_results)

# Sauvegarder les résultats dans un fichier CSV
results_df.to_csv('url_categories_results.csv', index=False)

# Télécharger le fichier généré (si exécuté dans Google Colab)
files.download('url_categories_results.csv')
logger.info("Résultats sauvegardés dans 'url_categories_results.csv' et téléchargement initié.")


Logs :

Utilisation de logging pour suivre les étapes du traitement, incluant les informations sur les requêtes réussies et les erreurs rencontrées.
Gestion des Erreurs :

Les exceptions liées aux requêtes sont capturées et loggées avec requests.exceptions.RequestException.
Les erreurs de réponse non 200 OK sont également loggées pour permettre un suivi précis des problèmes.
Limite de Requêtes :

La limite de 4 requêtes par seconde est gérée par des sleep de 0.25 seconde entre les requêtes et un sleep de 1.5 secondes après chaque 4 requêtes pour assurer le respect des limites.
Rapports de Progression :

Les logs fournissent un suivi détaillé des lots d'URLs traités, ce qui est utile pour diagnostiquer où le processus pourrait ralentir ou échouer.

Annexe 1: Test Swagger

In [ ]:
import requests

# URL de l'API
url = 'https://stib-mivb.eu.goskope.com/api/v2/nsiq/urllookup'

# Remplacez par votre token d'autorisation
api_token = '1aebabf6eb47804ae59ddaaafc23d101'

# En-têtes de la requête
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
    'Netskope-Api-Token': api_token  # Ajout du token d'autorisation
}

# Corps de la requête
data = {
    "query": {
        "disable_dns_lookup": True,
        "urls": [
            "https://www.netskope.com",
            "https://www.google.com",
            "https://www.airbnb.com"
        ]
    }
}

# Envoi de la requête POST
response = requests.post(url, headers=headers, json=data)

# Check the HTTP status code
if response.status_code == 200:
    # Successful response, proceed to decode JSON
    print(response.json())
else:
    # Handle non-200 responses
    print(f"Request failed with status code: {response.status_code}")
    print(response.text)  # Print the raw response content for debugging
